In [ ]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, Aer, execute, IBMQ, transpile, assemble
from qiskit.visualization import plot_histogram, plot_bloch_multivector
from qiskit.tools.jupyter import *
from qiskit.utils import QuantumInstance
from qiskit import Aer
from qiskit.algorithms import QAOA
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit_optimization.applications import VertexCover
import pandas as pd
import networkx as nx

In [ ]:
from typing import Dict, List, Optional, Union

import networkx as nx
import numpy as np
from docplex.mp.model import Model
from qiskit_optimization.algorithms import OptimizationResult
from qiskit_optimization.problems.quadratic_program import QuadraticProgram
from qiskit_optimization.translators import from_docplex_mp
from qiskit_optimization.applications.graph_optimization_application import GraphOptimizationApplication

In [ ]:
IBMQ.save_account("7ec7c3d9c61b02577a80462aeed74b169ce4defa242d2b2fc5031cabfac5f8199274a09d822b1dcd65125930a20f7c7c5f147e4785837efcc84764640599e462", overwrite = True)

In [ ]:
IBMQ.load_account()
provider = IBMQ.get_provider()

In [ ]:
data = pd.read_csv('data/hero_network.csv')
heroes = {'MEDUSA/MEDUSALITH AM', 'WOLVERINE/LOGAN ','SCARLET WITCH/WANDA ', 'ARAGORN','OVERMIND/GROM'}

In [ ]:
relations = data[(data.hero1.isin(heroes)) & (data.hero2.isin(heroes))]
relations.info()

In [ ]:
relations = pd.DataFrame(np.sort(relations.values, axis=1), columns=relations.columns).drop_duplicates(subset=['hero1', 'hero2'])

In [ ]:
graph = nx.from_pandas_edgelist(relations, source='hero1', target='hero2')

In [ ]:
class VertexCover(GraphOptimizationApplication):
    """Optimization application for the "vertex cover" problem based on a NetworkX graph."""
    def to_quadratic_program(self) -> QuadraticProgram:
        """Convert a vertex cover instance into a:class:`~qiskit_optimization.problems.QuadraticProgram`
        Returns:
        The :class:`~qiskit_optimization.problems.QuadraticProgram` created
        from the vertex cover instance.
        """
        mdl = Model(name="Vertex cover")
        n = self._graph.number_of_nodes()
        x = {i: mdl.binary_var(name=f"x_{i}") for i in range(n)}
        objective = mdl.sum(x[i] for i in x)
        for w, v in self._graph.edges:
            mdl.add_constraint(x[w] + x[v] >= 1)
        mdl.minimize(objective)
        op = from_docplex_mp(mdl)
        return op
    
    def interpret(self, result: Union[OptimizationResult, np.ndarray]) -> List[int]:
        """Interpret a result as a list of node indices
        Args:
        result : The calculated result of the problem
        Returns: A list of node indices whose corresponding variable is 1
        """
        x = self._result_to_x(result)
        vertex_cover = []
        for i, value in enumerate(x):
            if value:
                vertex_cover.append(i)
        return vertex_cover
    def _draw_result(self, result: Union[OptimizationResult, np.ndarray], pos: Optional[Dict[int, np.ndarray]] = None,) -> None:
        """Draw the result with colors
        Args:
        result : The calculated result for the problem
        pos: The positions of nodes
        """
        x = self._result_to_x(result)
        nx.draw(self._graph, node_color=self._node_colors(x), pos=pos, with_labels=True)

    def _node_colors(self, x: np.ndarray) -> List[str]:
        # Return a list of strings for draw.\
        # Color a node with red when the corresponding variable is 1.
        # Otherwise color it with dark gray.
        return ["r" if x[node[0]] else "darkgrey" for node in self._graph.nodes]

In [ ]:
class WeightedVertexCover(VertexCover):
    """Optimization application for the "weighted vertex cover" problem based on a NetworkX graph."""
    def to_quadratic_program(self) -> QuadraticProgram:
        mdl = Model(name="Weighted Vertex cover")
        n = self._graph.number_of_nodes()
        x = {i: mdl.binary_var(name=f"x_{i}") for i in range(n)}
        nodes = dict(enumerate(self._graph.nodes))
        lnodes = list(self._graph.nodes)
        
        max_weight= max(self._graph.nodes[node]['weight'] for node in lnodes)
        
        for w, v in self._graph.edges:
            self._graph.edges[w, v].setdefault("weight", 1)
            objective = mdl.sum(x[i] * (max_weight-self._graph.nodes[nodes[i]]['weight']) for i in x)
        
        for w, v in self._graph.edges:
            mdl.add_constraint(x[lnodes.index(w)] + x[lnodes.index(v)] >= 1)
            mdl.minimize(objective)
            op = from_docplex_mp(mdl)
        
        return op
    
    def _node_colors(self, x: np.ndarray) -> List[str]:
        nodes = list(dict(enumerate(self._graph.nodes)).items())

        # Return a list of strings for draw.
        # Color a node with red when the corresponding variable is 1.
        # Otherwise color it with dark gray.
        return ["r" if x[node[0]] else "darkgrey" for node in nodes]

In [ ]:
count_series1 = data.groupby(['hero1']).size()
count_series2 = data.groupby(['hero2']).size()
appearences = count_series1.add(count_series2, fill_value=0)
for hero in heroes:
    graph.nodes[hero]["weight"] = appearences[hero]

weighted_vertex_cover = WeightedVertexCover(graph)

In [ ]:
%qiskit_job_watcher

In [ ]:
# get the real Quito backend
quito_backend = provider.get_backend('ibmq_quito')
qp = weighted_vertex_cover.to_quadratic_program()
quantum_instance = QuantumInstance(backend=quito_backend,)
qaoa = QAOA(reps = 3, quantum_instance=quantum_instance)

# create minimum eigen optimizer based on qaoa
qaoa_optimizer = MinimumEigenOptimizer(qaoa)

# solve quadratic program
result = qaoa_optimizer.solve(qp)
print("solution:", weighted_vertex_cover.interpret(result))
print("time:", result.min_eigen_solver_result.optimizer_time)
print("cost function evals: ", result.min_eigen_solver_result.cost_function_evals)

In [ ]:
%qiskit_disable_job_watcher

In [ ]:
def callback(cnt, params, mean, stdv):
    print (cnt, params, mean, stdv)

qaoa = QAOA(reps=1, quantum_instance=quantum_instance, callback=callback,initial_point= [-4.57916767e+08, -2.78493997e+08])

In [ ]:
# loading the problem
qp = weighted_vertex_cover.to_quadratic_program()

# specifying the backend and the quantum instance
backend = Aer.get_backend("statevector_simulator")
quantum_instance = QuantumInstance(backend)

# Specify the solving algorithm
qaoa = QAOA(reps=2, quantum_instance=quantum_instance)

# create minimum eigen optimizer based on qaoa
qaoa_optimizer = MinimumEigenOptimizer(qaoa)

# solve quadratic program
result = qaoa_optimizer.solve(qp)